In [1]:
from data.process import *

2024-07-19 18:53:34.975125: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-19 18:53:34.982475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 18:53:34.993458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 18:53:34.993482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 18:53:35.000407: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_for_catboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
print(X_train.shape)

(11389751, 10)


In [4]:
X_train.dtypes

Gender                  object
Age                      int64
Driving_License          int64
Region_Code              int64
Previously_Insured       int64
Vehicle_Age             object
Vehicle_Damage          object
Annual_Premium           int64
Policy_Sales_Channel     int64
Vintage                  int64
dtype: object

In [5]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu
import numpy as np
from model.catboost_opt import *
import optuna
import pprint

In [6]:
cat_features = [i for i in range(X_train.shape[1])]
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [7]:
dataset = cb.Pool(X_train, label=Y_train, cat_features=cat_features)

In [8]:
tuned_params = {
    **const_params,
    'learning_rate': 0.095,
    'depth': 8,
    'bagging_temperature': 0.8,
    'l2_leaf_reg': 0.00255,
    'random_strength': 0,
    'border_count': 32,
    'n_estimators': 3000,
    'early_stopping_rounds': 200,
    'auto_class_weights': 'None',
    'leaf_estimation_iterations': 5,
    # overriding
    'eval_metric': 'AUC', 
    'custom_metric': ['AUC'],
}

In [9]:
model = cb.CatBoostClassifier(**tuned_params, cat_features=cat_features)
model.fit(dataset, verbose=False, eval_set=(X_dev, Y_dev), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


In [10]:
# model.save_model('catboost_model.json', format='json', pool=dataset)

In [11]:

model.save_model('catboost_model_categorical.cbm')

In [18]:
test_pool = cb.Pool(test_df, cat_features=X_train.columns.values)
cb_output = model.predict(test_df, prediction_type='Probability', ntree_end=model.get_best_iteration())

In [19]:
# Two-dimensional numpy.ndarray of shape (number_of_objects, number_of_classes) with the probability for every class for each object.
print(cb_output)

[[9.94684248e-01 5.31575201e-03]
 [3.53597029e-01 6.46402971e-01]
 [7.70607733e-01 2.29392267e-01]
 ...
 [9.99880462e-01 1.19537963e-04]
 [3.93264156e-01 6.06735844e-01]
 [9.99935185e-01 6.48149063e-05]]


In [20]:
# since it is binary classification, we only need the second column
cb_output = cb_output[:, 1]